In [ ]:
# Imports applicable to all scraping scripts

In [1]:
from bs4 import BeautifulSoup as BS
import requests
from selenium import webdriver
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd
import numpy as np
import datetime
from time import sleep
import os

# Results

In [105]:
for tr in table.find('thead').findAll('tr'):
    headers = [c.text for c in tr.findAll('th')]
headers.append('Date')

In [2]:
headers = ['#', 'Season', 'Name', 'Team', 
            'G', 'PA', 'AB', 'H', '1B', '2B', 
            '3B', 'HR', 'R', 'RBI', 'BB', 'IBB', 
            'SO', 'HBP', 'SF', 'SH', 'GDP', 'SB', 
            'CS', 'AVG', 'Date']

In [3]:
# Create list of days to iterate over
start = datetime.date(year = 2015, month = 4, day = 5)
end = datetime.date(year = 2015, month = 10, day = 4)
date_range = []
while start <= end:
    date_range.append(start)
    start += datetime.timedelta(days = 1)

# Create master dataframe and open firefox
master_df = pd.DataFrame(columns = headers)
binary = FirefoxBinary(r'C:\Program Files (x86)\Mozilla Firefox\firefox.exe')
browser = webdriver.Firefox(firefox_binary=binary)

# iterate through all days in range, and enter into url
for day in date_range:
    page = 0
    url_list = ['http://www.fangraphs.com/leaderssplits.aspx?splitArr=&strgroup=season&statgroup=1&startDate=',
           str(day),
                '&endDate=',
           str(day),
            '&filter=&position=B&statType=player&autoPt=false&sort=22,1&pg=',
           str(page)]
    url = "".join(url_list)
    browser.get(url)   
    sleep(1)
    
    #pull html table from first page
    soup = BS(browser.page_source, "lxml")
    table = soup.find(attrs = {'class':"table-splits"})
    body = table.find('tbody')
    if body != None:
        rows = body.findAll('tr')
    else:
        continue
    
    #parses data and appends to master dataframe
    data = []
    for tr in rows:
        cols = tr.findAll('td')
        row_text = [c.text for c in cols]
        row_text.append(day)
        data.append(row_text)
    data_df = pd.DataFrame(data, columns=headers)
    master_df = master_df.append(data_df)
    
    #iterate through all pages for given day
    page = int(soup.find('span', attrs = {'class':"table-control-total"}).text)
    for p in range(1,page):
        
        #scroll to bottom of page to prevent ad intereference
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        browser.find_element_by_class_name('material-icons' and 'next').click()
        sleep(1)        
        
        #parses data and appends to master dataframe
        data = []
        soup = BS(browser.page_source, "lxml")
        table = soup.find(attrs = {'class':"table-splits"})
        body = table.find('tbody')
        if body != None:
            rows = body.findAll('tr')
        else:
            continue
        for tr in rows:
            cols = tr.findAll('td')
            row_text = [c.text for c in cols]
            row_text.append(day)
            data.append(row_text)
        data_df = pd.DataFrame(data, columns=headers)
        master_df = master_df.append(data_df)

In [5]:

master_df.to_csv('data/hitting/results_master.csv')

In [28]:
master_df = pd.DataFrame(columns = headers)
for file in os.listdir():
    if file.endswith('csv'):
        master_df = master_df.append(pd.read_csv(file, index_col = 0))
master_df.to_csv('results_master.csv')

# Advanced Stats

## Home

In [6]:
#get headers
url = "http://www.fangraphs.com/leaderssplits.aspx?splitArr=7&strgroup=season&statgroup=2&startDate=2016-04-03&endDate=2016-06-10&filter=&position=B&statType=player&autoPt=false&sort=16,1&pg=0"
binary = FirefoxBinary(r'C:\Program Files (x86)\Mozilla Firefox\firefox.exe')
browser = webdriver.Firefox(firefox_binary=binary)
browser.get(url)   
sleep(1)
soup = BS(browser.page_source, "lxml")
table = soup.find(attrs = {'class':"table-splits"})
for tr in table.find('thead').findAll('tr'):
    headers = [c.text for c in tr.findAll('th')]
headers.append('Date')

In [8]:
headers = ['#',
 'Season',
 'Name',
 'Team',
 'PA',
 'BB%',
 'K%',
 'BB/K',
 'AVG',
 'OBP',
 'SLG',
 'OPS',
 'ISO',
 'BABIP',
 'wRC',
 'wRAA',
 'wOBA',
 'wRC+',
 'Date']

### Season

In [11]:
# Create list of days to iterate over
start = datetime.date(year = 2015, month = 4, day = 5)
end = datetime.date(year = 2015, month = 10, day = 4)
date_range = []
while start <= end:
    date_range.append(start)
    start += datetime.timedelta(days = 1)

# Create master dataframe and open firefox
master_df = pd.DataFrame(columns = headers)
binary = FirefoxBinary(r'C:\Program Files (x86)\Mozilla Firefox\firefox.exe')
browser = webdriver.Firefox(firefox_binary=binary)

# iterate through all days in range, and enter in url
for day in date_range:
    page = 0
    url_list = ['http://www.fangraphs.com/leaderssplits.aspx?splitArr=7&strgroup=season&statgroup=2&startDate=2015-04-05&endDate=',
           str(day),
            '&filter=&position=B&statType=player&autoPt=false&sort=16,1&pg=',
           str(page)]
    url = "".join(url_list)
    browser.get(url)   
    table = None
    
    #pull html table from first page
    while table == None:
        soup = BS(browser.page_source, "lxml")
        table = soup.find(attrs = {'class':"table-splits"})
        sleep(1)
    body = table.find('tbody')
    if body != None:
        rows = body.findAll('tr')
    else:
        continue
    
    #parses data and appends to master dataframe
    data = []
    for tr in rows:
        cols = tr.findAll('td')
        row_text = [c.text for c in cols]
        row_text.append(day)
        data.append(row_text)
    data_df = pd.DataFrame(data, columns=headers)
    master_df = master_df.append(data_df)
    
    #iterate through all pages for given day
    page = int(soup.find('span', attrs = {'class':"table-control-total"}).text)
    for p in range(1,page):
        
        #scroll to bottom of page to prevent ad intereference
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        browser.find_element_by_class_name('material-icons' and 'next').click()
        sleep(1)             
        
        #parses data and appends to master dataframe
        data = []
        soup = BS(browser.page_source, "lxml")
        table = soup.find(attrs = {'class':"table-splits"})
        body = table.find('tbody')
        if body != None:
            rows = body.findAll('tr')
        else:
            continue
        for tr in rows:
            cols = tr.findAll('td')
            row_text = [c.text for c in cols]
            row_text.append(day)
            data.append(row_text)
        data_df = pd.DataFrame(data, columns=headers)
        master_df = master_df.append(data_df)

In [12]:
master_df.to_csv('data/hitting/season_home_master.csv')

In [38]:
master_df = pd.DataFrame(columns = headers)
for file in os.listdir('data/hitting/'):
    if file.startswith('season_home'):
        master_df = master_df.append(pd.read_csv('data/hitting/'+file, index_col = 0))
master_df.to_csv('data/hitting/season_home_master.csv')

## Away
### Season

In [15]:
# Create list of days to iterate over
start = datetime.date(year = 2015, month = 4, day = 5)
end = datetime.date(year = 2015, month = 10, day = 4)
date_range = []
while start <= end:
    date_range.append(start)
    start += datetime.timedelta(days = 1)

# Create master dataframe and open firefox
master_df = pd.DataFrame(columns = headers)
binary = FirefoxBinary(r'C:\Program Files (x86)\Mozilla Firefox\firefox.exe')
browser = webdriver.Firefox(firefox_binary=binary)

# iterate through all days in range, and enter in url
for day in date_range:
    page = 0
    url_list = ['http://www.fangraphs.com/leaderssplits.aspx?splitArr=8&strgroup=season&statgroup=2&startDate=2015-04-05&endDate=',
           str(day),
            '&filter=&position=B&statType=player&autoPt=false&sort=16,1&pg=',
           str(page)]
    url = "".join(url_list)
    browser.get(url)   
    table = None
    
    #pull html table from first page
    while table == None:
        soup = BS(browser.page_source, "lxml")
        table = soup.find(attrs = {'class':"table-splits"})
        sleep(1)
    body = table.find('tbody')
    if body != None:
        rows = body.findAll('tr')
    else:
        continue
    
    #parses data and appends to master dataframe
    data = []
    for tr in rows:
        cols = tr.findAll('td')
        row_text = [c.text for c in cols]
        row_text.append(day)
        data.append(row_text)
    data_df = pd.DataFrame(data, columns=headers)
    master_df = master_df.append(data_df)
    
    #iterate through all pages for given day
    page = int(soup.find('span', attrs = {'class':"table-control-total"}).text)
    for p in range(1,page):
        
        #scroll to bottom of page to prevent ad intereference
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")        
        
        browser.find_element_by_class_name('material-icons' and 'next').click()
        sleep(1)
        
        #parses data and appends to master dataframe
        data = []
        soup = BS(browser.page_source, "lxml")
        table = soup.find(attrs = {'class':"table-splits"})
        body = table.find('tbody')
        if body != None:
            rows = body.findAll('tr')
        else:
            continue
        for tr in rows:
            cols = tr.findAll('td')
            row_text = [c.text for c in cols]
            row_text.append(day)
            data.append(row_text)
        data_df = pd.DataFrame(data, columns=headers)
        master_df = master_df.append(data_df)

In [16]:
master_df.to_csv('data/hitting/season_away_master.csv')

In [57]:
master_df = pd.DataFrame(columns = headers)
for file in os.listdir('data/hitting/'):
    if file.startswith('season_away'):
        master_df = master_df.append(pd.read_csv('data/hitting/'+file, index_col = 0))
master_df.to_csv('data/hitting/season_away_master.csv')

## Lefties
### Season

In [21]:
# Create list of days to iterate over
start = datetime.date(year = 2015, month = 7, day = 2)
end = datetime.date(year = 2015, month = 10, day = 4)
date_range = []
while start <= end:
    date_range.append(start)
    start += datetime.timedelta(days = 1)

# Create master dataframe and open firefox
master_df = pd.DataFrame(columns = headers)
binary = FirefoxBinary(r'C:\Program Files (x86)\Mozilla Firefox\firefox.exe')
browser = webdriver.Firefox(firefox_binary=binary)

# iterate through all days in range, and enter in url
for day in date_range:
    page = 0
    url_list = ['http://www.fangraphs.com/leaderssplits.aspx?splitArr=1&strgroup=season&statgroup=2&startDate=2015-04-05&endDate=',
           str(day),
            '&filter=&position=B&statType=player&autoPt=false&sort=16,1&pg=',
           str(page)]
    url = "".join(url_list)
    browser.get(url)   
    table = None
    
    #pull html table from first page
    while table == None:
        soup = BS(browser.page_source, "lxml")
        table = soup.find(attrs = {'class':"table-splits"})
        sleep(1)
    body = table.find('tbody')
    if body != None:
        rows = body.findAll('tr')
    else:
        continue
    
    #parses data and appends to master dataframe
    data = []
    for tr in rows:
        cols = tr.findAll('td')
        row_text = [c.text for c in cols]
        row_text.append(day)
        data.append(row_text)
    data_df = pd.DataFrame(data, columns=headers)
    master_df = master_df.append(data_df)
    
    #iterate through all pages for given day
    page = int(soup.find('span', attrs = {'class':"table-control-total"}).text)
    for p in range(1,page):
        
        #scroll to bottom of page to prevent ad interference
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        browser.find_element_by_class_name('material-icons' and 'next').click()
        sleep(1)
        
        #parses data and appends to master dataframe
        data = []
        soup = BS(browser.page_source, "lxml")
        table = soup.find(attrs = {'class':"table-splits"})
        body = table.find('tbody')
        if body != None:
            rows = body.findAll('tr')
        else:
            continue
        for tr in rows:
            cols = tr.findAll('td')
            row_text = [c.text for c in cols]
            row_text.append(day)
            data.append(row_text)
        data_df = pd.DataFrame(data, columns=headers)
        master_df = master_df.append(data_df)

In [22]:
master_df

,#,Season,Name,Team,PA,BB%,K%,BB/K,AVG,OBP,SLG,OPS,ISO,BABIP,wRC,wRAA,wOBA,wRC+,Date
0,1,2015,Kyle Blanks,TEX,27,0.0%,25.9%,0.0,.333,.333,.778,1.111,.444,.353,6,3.3,.467,199,2015-07-02
1,2,2015,Jason Marquis,CIN,6,0.0%,33.3%,0.0,.000,.000,.000,.000,.000,.000,-1,-1.5,.000,-100,2015-07-02
2,3,2015,Will Venable,SDP,21,4.8%,28.6%,0.2,.250,.286,.300,.586,.050,.357,1,-0.9,.260,64,2015-07-02
3,4,2015,Bartolo Colon,NYM,7,0.0%,42.9%,0.0,.143,.143,.143,.286,.000,.250,-0,-1.0,.126,-27,2015-07-02
4,5,2015,Victor Martinez,DET,45,8.9%,4.4%,2.0,.410,.467,.564,1.031,.154,.405,9,4.4,.436,181,2015-07-02
5,6,2015,Juan Uribe,2 Tms,39,10.3%,17.9%,0.6,.286,.359,.543,.902,.257,.280,6,2.1,.380,145,2015-07-02
6,7,2015,A.J. Burnett,PIT,4,0.0%,50.0%,0.0,.250,.250,.250,.500,.000,.500,0,-0.3,.220,36,2015-07-02
7,8,2015,Carlos Beltran,NYY,75,8.0%,21.3%,0.4,.239,.307,.403,.710,.164,.280,8,-0.6,.303,90,2015-07-02
8,9,2015,Brandon Barnes,COL,27,7.4%,25.9%,0.3,.200,.259,.200,.459,.000,.278,1,-2.1,.214,12,2015-07-02
9,10,2015,Adrian Beltre,TEX,84,4.8%,9.5%,0.5,.263,.298,.450,.748,.188,.261,10,0.3,.317,96,2015-07-02


In [23]:
master_df.append(master_df1).to_csv('data/hitting/season_lefties_master.csv')

In [75]:
master_df.to_csv('data/hitting/season_lefties_09_02_to_10_02.csv')

In [76]:
master_df = pd.DataFrame(columns = headers)
for file in os.listdir('data/hitting/'):
    if file.startswith('season_lefties'):
        master_df = master_df.append(pd.read_csv('data/hitting/'+file, index_col = 0))
master_df.to_csv('data/hitting/season_lefties_master.csv')

## Rightes
### Season

In [26]:
# Create list of days to iterate over
start = datetime.date(year = 2015, month = 4, day = 5)
end = datetime.date(year = 2015, month = 10, day = 4)
date_range = []
while start <= end:
    date_range.append(start)
    start += datetime.timedelta(days = 1)

# Create master dataframe and open firefox
master_df = pd.DataFrame(columns = headers)
binary = FirefoxBinary(r'C:\Program Files (x86)\Mozilla Firefox\firefox.exe')
browser = webdriver.Firefox(firefox_binary=binary)

# iterate through all days in range, and enter in url
for day in date_range:
    page = 0
    url_list = ['http://www.fangraphs.com/leaderssplits.aspx?splitArr=2&strgroup=season&statgroup=2&startDate=2015-04-05&endDate=',
           str(day),
            '&filter=&position=B&statType=player&autoPt=false&sort=16,1&pg=',
           str(page)]
    url = "".join(url_list)
    browser.get(url)   
    table = None
    
    #pull html table from first page
    while table == None:
        soup = BS(browser.page_source, "lxml")
        table = soup.find(attrs = {'class':"table-splits"})
        sleep(1)
    body = table.find('tbody')
    if body != None:
        rows = body.findAll('tr')
    else:
        continue
    
    #parses data and appends to master dataframe
    data = []
    for tr in rows:
        cols = tr.findAll('td')
        row_text = [c.text for c in cols]
        row_text.append(day)
        data.append(row_text)
    data_df = pd.DataFrame(data, columns=headers)
    master_df = master_df.append(data_df)
    
    #iterate through all pages for given day
    page = int(soup.find('span', attrs = {'class':"table-control-total"}).text)
    for p in range(1,page):
        
        #scroll to bottom of page to prevent ad interference
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")        
        
        browser.find_element_by_class_name('material-icons' and 'next').click()
        sleep(1)
        
        #parses data and appends to master dataframe
        data = []
        soup = BS(browser.page_source, "lxml")
        table = soup.find(attrs = {'class':"table-splits"})
        body = table.find('tbody')
        if body != None:
            rows = body.findAll('tr')
        else:
            continue
        for tr in rows:
            cols = tr.findAll('td')
            row_text = [c.text for c in cols]
            row_text.append(day)
            data.append(row_text)
        data_df = pd.DataFrame(data, columns=headers)
        master_df = master_df.append(data_df)

In [27]:
master_df.to_csv('data/hitting/season_righties_master.csv')

In [96]:
master_df = pd.DataFrame(columns = headers)
for file in os.listdir('data/hitting/'):
    if file.startswith('season_righties'):
        master_df = master_df.append(pd.read_csv('data/hitting/'+file, index_col = 0))
master_df.to_csv('data/hitting/season_righties_master.csv')

## Past 7 Days

In [28]:
datetime.date(year = 2015, month = 10, day = 4) - datetime.timedelta(days = 7)

datetime.date(2015, 9, 27)

In [29]:
# Create list of days to iterate over
start = datetime.date(year = 2015, month = 4, day = 5)
end = datetime.date(year = 2015, month = 9, day = 27)
date_range = []
while start <= end:
    date_range.append(start)
    start += datetime.timedelta(days = 1)

# Create master dataframe and open firefox
master_df = pd.DataFrame(columns = headers)
binary = FirefoxBinary(r'C:\Program Files (x86)\Mozilla Firefox\firefox.exe')
browser = webdriver.Firefox(firefox_binary=binary)

# iterate through all days in range, and enter in url
for day in date_range:
    page = 0
    url_list = ['http://www.fangraphs.com/leaderssplits.aspx?splitArr=&strgroup=season&statgroup=2&startDate=',
           str(day),'&endDate=',str(day + datetime.timedelta(days = 7)),
            '&filter=&position=B&statType=player&autoPt=false&sort=16,1&pg=',
           str(page)]
    url = "".join(url_list)
    browser.get(url)   
    table = None
    
    #pull html table from first page
    while table == None:
        soup = BS(browser.page_source, "lxml")
        table = soup.find(attrs = {'class':"table-splits"})
        sleep(1)
    body = table.find('tbody')
    if body != None:
        rows = body.findAll('tr')
    else:
        continue
    
    #parses data and appends to master dataframe
    data = []
    for tr in rows:
        cols = tr.findAll('td')
        row_text = [c.text for c in cols]
        row_text.append(day + datetime.timedelta(days = 7))
        data.append(row_text)
    data_df = pd.DataFrame(data, columns=headers)
    master_df = master_df.append(data_df)
    
    #iterate through all pages for given day
    page = int(soup.find('span', attrs = {'class':"table-control-total"}).text)
    for p in range(1,page):
        
        #scroll to bottom of page to prevent ad interference
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")          
        
        browser.find_element_by_class_name('material-icons' and 'next').click()
        sleep(1)
        
        #parses data and appends to master dataframe
        data = []
        soup = BS(browser.page_source, "lxml")
        table = soup.find(attrs = {'class':"table-splits"})
        body = table.find('tbody')
        if body != None:
            rows = body.findAll('tr')
        else:
            continue
        for tr in rows:
            cols = tr.findAll('td')
            row_text = [c.text for c in cols]
            row_text.append(day + datetime.timedelta(days = 7))
            data.append(row_text)
        data_df = pd.DataFrame(data, columns=headers)
        master_df = master_df.append(data_df)

In [30]:
master_df.to_csv('data/hitting/week_master.csv')

In [119]:
master_df = pd.DataFrame(columns = headers)
for file in os.listdir('data/hitting/'):
    if file.startswith('week'):
        master_df = master_df.append(pd.read_csv('data/hitting/'+file, index_col = 0))
master_df.to_csv('data/hitting/week_master.csv')

## Batted Balls Season

In [253]:
binary = FirefoxBinary(r'C:\Program Files (x86)\Mozilla Firefox\firefox.exe')
browser = webdriver.Firefox(firefox_binary=binary)
page = 0
url_list = ['http://www.fangraphs.com/leaderssplits.aspx?splitArr=&strgroup=season&statgroup=3&startDate=2016-04-03&endDate=',
       str(day),
        '&filter=&position=B&statType=player&autoPt=false&sort=17,1&pg=',
       str(page)]
url = "".join(url_list)
browser.get(url)
table = None
    
#pull html table from first page
while table == None:
    soup = BS(browser.page_source, "lxml")
    table = soup.find(attrs = {'class':"table-splits"})
    sleep(1)

In [190]:
for tr in table.find('thead').findAll('tr'):
    headers = [c.text for c in tr.findAll('th')]
headers.append('Date')

In [32]:
headers = ['#',
 'Season',
 'Name',
 'Team',
 'PA',
 'GB/FB',
 'LD%',
 'GB%',
 'FB%',
 'IFFB%',
 'HR/FB',
 'IFH%',
 'BUH%',
 'Pull%',
 'Cent%',
 'Oppo%',
 'Soft%',
 'Med%',
 'Hard%',
 'Date']

In [34]:
# Create list of days to iterate over
start = datetime.date(year = 2015, month = 4, day = 5)
end = datetime.date(year = 2015, month = 10, day = 4)
date_range = []
while start <= end:
    date_range.append(start)
    start += datetime.timedelta(days = 1)

# Create master dataframe and open firefox
master_df = pd.DataFrame(columns = headers)
binary = FirefoxBinary(r'C:\Program Files (x86)\Mozilla Firefox\firefox.exe')
browser = webdriver.Firefox(firefox_binary=binary)

# iterate through all days in range, and enter in url
for day in date_range:
    page = 0
    url_list = ['http://www.fangraphs.com/leaderssplits.aspx?splitArr=&strgroup=season&statgroup=3&startDate=2015-04-05&endDate=',
           str(day),
            '&filter=&position=B&statType=player&autoPt=false&sort=17,1&pg=',
           str(page)]
    url = "".join(url_list)
    browser.get(url)   
    table = None
    
    #pull html table from first page
    while table == None:
        soup = BS(browser.page_source, "lxml")
        table = soup.find(attrs = {'class':"table-splits"})
        sleep(1)
    body = table.find('tbody')
    if body != None:
        rows = body.findAll('tr')
    else:
        continue
    
    #parses data and appends to master dataframe
    data = []
    for tr in rows:
        cols = tr.findAll('td')
        row_text = [c.text for c in cols]
        row_text.append(day)
        data.append(row_text)
    data_df = pd.DataFrame(data, columns=headers)
    master_df = master_df.append(data_df)
    
    #close stupid ad getting in the way
    
    
    #iterate through all pages for given day
    page = int(soup.find('span', attrs = {'class':"table-control-total"}).text)
    for p in range(1,page):
        
        #scroll to bottom of page to prevent ad intereference
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        #click next button
        browser.find_element_by_class_name('material-icons' and 'next').click()
        sleep(1)
        
        #parses data and appends to master dataframe
        data = []
        soup = BS(browser.page_source, "lxml")
        table = soup.find(attrs = {'class':"table-splits"})
        body = table.find('tbody')
        if body != None:
            rows = body.findAll('tr')
        else:
            continue
        for tr in rows:
            cols = tr.findAll('td')
            row_text = [c.text for c in cols]
            row_text.append(day)
            data.append(row_text)
        data_df = pd.DataFrame(data, columns=headers)
        master_df = master_df.append(data_df)

In [36]:
master_df.to_csv('data/hitting/season_batted_balls_master.csv')

In [284]:
master_df = pd.DataFrame(columns = headers)
for file in os.listdir('data/hitting/'):
    if file.startswith('season_batted_balls'):
        master_df = master_df.append(pd.read_csv('data/hitting/'+file, index_col = 0))
master_df.to_csv('data/hitting/season_batted_balls_master.csv')

## Batted Balls 7 days

In [37]:
headers = ['#',
 'Season',
 'Name',
 'Team',
 'PA',
 'GB/FB',
 'LD%',
 'GB%',
 'FB%',
 'IFFB%',
 'HR/FB',
 'IFH%',
 'BUH%',
 'Pull%',
 'Cent%',
 'Oppo%',
 'Soft%',
 'Med%',
 'Hard%',
 'Date']

In [50]:
# Create list of days to iterate over
start = datetime.date(year = 2015, month = 6, day = 9)
end = datetime.date(year = 2015, month = 9, day = 27)
date_range = []
while start <= end:
    date_range.append(start)
    start += datetime.timedelta(days = 1)

# Create master dataframe and open firefox
master_df = pd.DataFrame(columns = headers)
binary = FirefoxBinary(r'C:\Program Files (x86)\Mozilla Firefox\firefox.exe')
browser = webdriver.Firefox(firefox_binary=binary)

# iterate through all days in range, and enter in url
for day in date_range:
    page = 0
    url_list = ['http://www.fangraphs.com/leaderssplits.aspx?splitArr=&strgroup=season&statgroup=3&startDate=',
           str(day),'&endDate=',str(day + datetime.timedelta(days = 7)),
            '&filter=&position=B&statType=player&autoPt=false&sort=17,1&pg=',
           str(page)]
    url = "".join(url_list)
    browser.get(url)   
    table = None
    
    #pull html table from first page
    while table == None:
        soup = BS(browser.page_source, "lxml")
        table = soup.find(attrs = {'class':"table-splits"})
        sleep(1)
    body = table.find('tbody')
    if body != None:
        rows = body.findAll('tr')
    else:
        continue
    
    #parses data and appends to master dataframe
    data = []
    for tr in rows:
        cols = tr.findAll('td')
        row_text = [c.text for c in cols]
        row_text.append(day + datetime.timedelta(days = 7))
        data.append(row_text)
    data_df = pd.DataFrame(data, columns=headers)
    master_df = master_df.append(data_df)
    
    #iterate through all pages for given day
    page = int(soup.find('span', attrs = {'class':"table-control-total"}).text)
    for p in range(1,page):
        
        #scroll to bottom of page to prevent ad intereference
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        browser.find_element_by_class_name('material-icons' and 'next').click()
        sleep(1)
        
        #parses data and appends to master dataframe
        data = []
        soup = BS(browser.page_source, "lxml")
        table = soup.find(attrs = {'class':"table-splits"})
        body = table.find('tbody')
        if body != None:
            rows = body.findAll('tr')
        else:
            continue
        for tr in rows:
            cols = tr.findAll('td')
            row_text = [c.text for c in cols]
            row_text.append(day + datetime.timedelta(days = 7))
            data.append(row_text)
        data_df = pd.DataFrame(data, columns=headers)
        master_df = master_df.append(data_df)

WebDriverException: Message: Failed to decode response from marionette


In [39]:
master_df.to_csv('data/hitting/past_7_batted_balls_master.csv')

In [337]:
master_df = pd.DataFrame(columns = headers)
for file in os.listdir('data/hitting/'):
    if file.startswith('past_7_batted_balls'):
        master_df = master_df.append(pd.read_csv('data/hitting/'+file, index_col = 0))
master_df.to_csv('data/hitting/past_7_batted_balls_master.csv')

## Team
### Season

In [377]:
binary = FirefoxBinary(r'C:\Program Files (x86)\Mozilla Firefox\firefox.exe')
browser = webdriver.Firefox(firefox_binary=binary)
page = 0
url_list = ['http://www.fangraphs.com/leaderssplits.aspx?splitArr=&strgroup=season&statgroup=2&startDate=2016-04-03&endDate=',
           str(day),
            '&filter=&position=B&statType=team&autoPt=false&sort=15,1&pg=',
           str(page)]
url = "".join(url_list)
browser.get(url)   
table = None

#pull html table from first page
while table == None:
    soup = BS(browser.page_source, "lxml")
    table = soup.find(attrs = {'class':"table-splits"})
    sleep(1)
for tr in table.find('thead').findAll('tr'):
    headers = [c.text for c in tr.findAll('th')]
headers.append('Date')

In [40]:
headers = ['#',
 'Season',
 'Team',
 'PA',
 'BB%',
 'K%',
 'BB/K',
 'AVG',
 'OBP',
 'SLG',
 'OPS',
 'ISO',
 'BABIP',
 'wRC',
 'wRAA',
 'wOBA',
 'wRC+',
 'Date']

In [41]:
# Create list of days to iterate over
start = datetime.date(year = 2015, month = 4, day = 5)
end = datetime.date(year = 2015, month = 10, day = 4)
date_range = []
while start <= end:
    date_range.append(start)
    start += datetime.timedelta(days = 1)

# Create master dataframe and open firefox
master_df = pd.DataFrame(columns = headers)
binary = FirefoxBinary(r'C:\Program Files (x86)\Mozilla Firefox\firefox.exe')
browser = webdriver.Firefox(firefox_binary=binary)

# iterate through all days in range, and enter in url
for day in date_range:
    page = 0
    url_list = ['http://www.fangraphs.com/leaderssplits.aspx?splitArr=&strgroup=season&statgroup=2&startDate=2015-04-05&endDate=',
           str(day),
            '&filter=&position=B&statType=team&autoPt=false&sort=15,1&pg=',
           str(page)]
    url = "".join(url_list)
    browser.get(url)   
    table = None
    
    #pull html table from first page
    while table == None:
        soup = BS(browser.page_source, "lxml")
        table = soup.find(attrs = {'class':"table-splits"})
        sleep(1)
    body = table.find('tbody')
    if body != None:
        rows = body.findAll('tr')
    else:
        continue
    
    #parses data and appends to master dataframe
    data = []
    for tr in rows:
        cols = tr.findAll('td')
        row_text = [c.text for c in cols]
        row_text.append(day)
        data.append(row_text)
    data_df = pd.DataFrame(data, columns=headers)
    master_df = master_df.append(data_df)
    
    #close stupid ad getting in the way
    
    
    #iterate through all pages for given day
    page = int(soup.find('span', attrs = {'class':"table-control-total"}).text)
    for p in range(1,page):
        
        #scroll to bottom of page to prevent ad intereference
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        #click next button
        browser.find_element_by_class_name('material-icons' and 'next').click()
        sleep(1)
        
        #parses data and appends to master dataframe
        data = []
        soup = BS(browser.page_source, "lxml")
        table = soup.find(attrs = {'class':"table-splits"})
        body = table.find('tbody')
        if body != None:
            rows = body.findAll('tr')
        else:
            continue
        for tr in rows:
            cols = tr.findAll('td')
            row_text = [c.text for c in cols]
            row_text.append(day)
            data.append(row_text)
        data_df = pd.DataFrame(data, columns=headers)
        master_df = master_df.append(data_df)

In [42]:
master_df.to_csv('data/hitting/team_season_master.csv')

In [385]:
master_df = pd.DataFrame(columns = headers)
for file in os.listdir('data/hitting/'):
    if file.startswith('team_season'):
        master_df = master_df.append(pd.read_csv('data/hitting/'+file, index_col = 0))
master_df.to_csv('data/hitting/team_season_master.csv')

### Past 7

In [43]:
# Create list of days to iterate over
start = datetime.date(year = 2015, month = 4, day = 5)
end = datetime.date(year = 2015, month = 9, day = 27)
date_range = []
while start <= end:
    date_range.append(start)
    start += datetime.timedelta(days = 1)

# Create master dataframe and open firefox
master_df = pd.DataFrame(columns = headers)
binary = FirefoxBinary(r'C:\Program Files (x86)\Mozilla Firefox\firefox.exe')
browser = webdriver.Firefox(firefox_binary=binary)

# iterate through all days in range, and enter in url
for day in date_range:
    page = 0
    url_list = ['http://www.fangraphs.com/leaderssplits.aspx?splitArr=&strgroup=season&statgroup=2&startDate=',
           str(day),'&endDate=',str(day + datetime.timedelta(days = 7)),
            '&filter=&position=B&statType=team&autoPt=false&sort=15,1&pg=',
           str(page)]
    url = "".join(url_list)
    browser.get(url)   
    table = None
    
    #pull html table from first page
    while table == None:
        soup = BS(browser.page_source, "lxml")
        table = soup.find(attrs = {'class':"table-splits"})
        sleep(1)
    body = table.find('tbody')
    if body != None:
        rows = body.findAll('tr')
    else:
        continue
    
    #parses data and appends to master dataframe
    data = []
    for tr in rows:
        cols = tr.findAll('td')
        row_text = [c.text for c in cols]
        row_text.append(day + datetime.timedelta(days = 7))
        data.append(row_text)
    data_df = pd.DataFrame(data, columns=headers)
    master_df = master_df.append(data_df)
    
    #iterate through all pages for given day
    page = int(soup.find('span', attrs = {'class':"table-control-total"}).text)
    for p in range(1,page):
        
        #scroll to bottom of page to prevent ad intereference
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        browser.find_element_by_class_name('material-icons' and 'next').click()
        sleep(1)
        
        #parses data and appends to master dataframe
        data = []
        soup = BS(browser.page_source, "lxml")
        table = soup.find(attrs = {'class':"table-splits"})
        body = table.find('tbody')
        if body != None:
            rows = body.findAll('tr')
        else:
            continue
        for tr in rows:
            cols = tr.findAll('td')
            row_text = [c.text for c in cols]
            row_text.append(day + datetime.timedelta(days = 7))
            data.append(row_text)
        data_df = pd.DataFrame(data, columns=headers)
        master_df = master_df.append(data_df)

In [44]:
master_df.to_csv('data/hitting/team_past7_master.csv')

In [392]:
master_df = pd.DataFrame(columns = headers)
for file in os.listdir('data/hitting/'):
    if file.startswith('team_past7'):
        master_df = master_df.append(pd.read_csv('data/hitting/'+file, index_col = 0))
master_df.to_csv('data/hitting/team_past7_master.csv')

# Pitching
## Home Season

In [122]:
for tr in table.find('thead').findAll('tr'):
    headers = [c.text for c in tr.findAll('th')]
headers.append('Date')

In [45]:
headers = ['#',
 'Season',
 'Name',
 'Team',
 'IP',
 'TBF',
 'K/9',
 'BB/9',
 'K/BB',
 'HR/9',
 'K%',
 'BB%',
 'K-BB%',
 'AVG',
 'WHIP',
 'BABIP',
 'LOB%',
 'FIP',
 'xFIP',
 'Date']

In [55]:
# Create list of days to iterate over
start = datetime.date(year = 2015, month = 4, day = 5)
end = datetime.date(year = 2015, month = 10, day = 4)
date_range = []
while start <= end:
    date_range.append(start)
    start += datetime.timedelta(days = 1)

# Create master dataframe and open firefox
master_df = pd.DataFrame(columns = headers)
binary = FirefoxBinary(r'C:\Program Files (x86)\Mozilla Firefox\firefox.exe')
browser = webdriver.Firefox(firefox_binary=binary)

# iterate through all days in range, and enter in url
for day in date_range:
    page = 0
    url_list = ['http://www.fangraphs.com/leaderssplits.aspx?splitArr=9&strgroup=season&statgroup=2&startDate=2015-04-05&endDate=',
           str(day),
            '&filter=&position=P&statType=player&autoPt=false&sort=17,-1&pg=',
           str(page)]
    url = "".join(url_list)
    browser.get(url)   
    table = None
    
    #pull html table from first page
    while table == None:
        soup = BS(browser.page_source, "lxml")
        table = soup.find(attrs = {'class':"table-splits"})
        sleep(1)
    body = table.find('tbody')
    if body != None:
        rows = body.findAll('tr')
    else:
        continue
    
    #parses data and appends to master dataframe
    data = []
    for tr in rows:
        cols = tr.findAll('td')
        row_text = [c.text for c in cols]
        row_text.append(day)
        data.append(row_text)
    data_df = pd.DataFrame(data, columns=headers)
    master_df = master_df.append(data_df)
    
    #iterate through all pages for given day
    page = int(soup.find('span', attrs = {'class':"table-control-total"}).text)
    for p in range(1,page):
        #scroll to bottom of page to prevent ad intereference
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        browser.find_element_by_class_name('material-icons' and 'next').click()
        sleep(1)
        
        #parses data and appends to master dataframe
        data = []
        soup = BS(browser.page_source, "lxml")
        table = soup.find(attrs = {'class':"table-splits"})
        body = table.find('tbody')
        if body != None:
            rows = body.findAll('tr')
        else:
            continue
        for tr in rows:
            cols = tr.findAll('td')
            row_text = [c.text for c in cols]
            row_text.append(day)
            data.append(row_text)
        data_df = pd.DataFrame(data, columns=headers)
        master_df = master_df.append(data_df)

In [57]:
master_df.to_csv('data/pitching/season_home_master.csv')

In [144]:
master_df = pd.DataFrame(columns = headers)
for file in os.listdir('data/pitching/'):
    if file.startswith('season_home'):
        master_df = master_df.append(pd.read_csv('data/pitching/'+file, index_col = 0))
master_df.to_csv('data/pitching/season_home_master.csv')

## Away

In [59]:
# Create list of days to iterate over
start = datetime.date(year = 2015, month = 4, day = 5)
end = datetime.date(year = 2015, month = 10, day = 4)
date_range = []
while start <= end:
    date_range.append(start)
    start += datetime.timedelta(days = 1)

# Create master dataframe and open firefox
master_df = pd.DataFrame(columns = headers)
binary = FirefoxBinary(r'C:\Program Files (x86)\Mozilla Firefox\firefox.exe')
browser = webdriver.Firefox(firefox_binary=binary)

# iterate through all days in range, and enter in url
for day in date_range:
    page = 0
    url_list = ['http://www.fangraphs.com/leaderssplits.aspx?splitArr=10&strgroup=season&statgroup=2&startDate=2015-04-05&endDate=',
           str(day),
            '&filter=&position=P&statType=player&autoPt=false&sort=17,-1&pg=',
           str(page)]
    url = "".join(url_list)
    browser.get(url)   
    table = None
    
    #pull html table from first page
    while table == None:
        soup = BS(browser.page_source, "lxml")
        table = soup.find(attrs = {'class':"table-splits"})
        sleep(1)
    body = table.find('tbody')
    if body != None:
        rows = body.findAll('tr')
    else:
        continue
    
    #parses data and appends to master dataframe
    data = []
    for tr in rows:
        cols = tr.findAll('td')
        row_text = [c.text for c in cols]
        row_text.append(day)
        data.append(row_text)
    data_df = pd.DataFrame(data, columns=headers)
    master_df = master_df.append(data_df)
    
    #iterate through all pages for given day
    page = int(soup.find('span', attrs = {'class':"table-control-total"}).text)
    for p in range(1,page):
        #scroll to bottom of page to prevent ad intereference
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")    
        
        browser.find_element_by_class_name('material-icons' and 'next').click()
        sleep(1)
        
        #parses data and appends to master dataframe
        data = []
        soup = BS(browser.page_source, "lxml")
        table = soup.find(attrs = {'class':"table-splits"})
        body = table.find('tbody')
        if body != None:
            rows = body.findAll('tr')
        else:
            continue
        for tr in rows:
            cols = tr.findAll('td')
            row_text = [c.text for c in cols]
            row_text.append(day)
            data.append(row_text)
        data_df = pd.DataFrame(data, columns=headers)
        master_df = master_df.append(data_df)

In [60]:
master_df.to_csv('data/pitching/season_away_master.csv')

In [164]:
master_df = pd.DataFrame(columns = headers)
for file in os.listdir('data/pitching/'):
    if file.startswith('season_away'):
        master_df = master_df.append(pd.read_csv('data/pitching/'+file, index_col = 0))
master_df.to_csv('data/pitching/season_away_master.csv')

## Past Three Starts

In [62]:
datetime.date(year = 2015, month = 10, day = 4) - datetime.timedelta(days = 17)

datetime.date(2015, 9, 17)

In [63]:
# Create list of days to iterate over
start = datetime.date(year = 2015, month = 3, day = 19)
end = datetime.date(year = 2015, month = 9, day = 17)
date_range = []
while start <= end:
    date_range.append(start)
    start += datetime.timedelta(days = 1)

# Create master dataframe and open firefox
master_df = pd.DataFrame(columns = headers)
binary = FirefoxBinary(r'C:\Program Files (x86)\Mozilla Firefox\firefox.exe')
browser = webdriver.Firefox(firefox_binary=binary)

# iterate through all days in range, and enter in url
for day in date_range:
    page = 0
    url_list = ['http://www.fangraphs.com/leaderssplits.aspx?splitArr=&strgroup=season&statgroup=2&startDate=',
           str(day),'&endDate=',str(day + datetime.timedelta(days = 17)),
            '&filter=&position=P&statType=player&autoPt=false&sort=17,-1&pg=',
           str(page)]
    url = "".join(url_list)
    browser.get(url)   
    table = None
    
    #pull html table from first page
    while table == None:
        soup = BS(browser.page_source, "lxml")
        table = soup.find(attrs = {'class':"table-splits"})
        sleep(1)
    body = table.find('tbody')
    if body != None:
        rows = body.findAll('tr')
    else:
        continue
    
    #parses data and appends to master dataframe
    data = []
    for tr in rows:
        cols = tr.findAll('td')
        row_text = [c.text for c in cols]
        row_text.append(day + datetime.timedelta(days = 17))
        data.append(row_text)
    data_df = pd.DataFrame(data, columns=headers)
    master_df = master_df.append(data_df)
    
    #iterate through all pages for given day
    page = int(soup.find('span', attrs = {'class':"table-control-total"}).text)
    for p in range(1,page):
        
        browser.find_element_by_class_name('material-icons' and 'next').click()
        sleep(1)
        
        #parses data and appends to master dataframe
        data = []
        soup = BS(browser.page_source, "lxml")
        table = soup.find(attrs = {'class':"table-splits"})
        body = table.find('tbody')
        if body != None:
            rows = body.findAll('tr')
        else:
            continue
        for tr in rows:
            cols = tr.findAll('td')
            row_text = [c.text for c in cols]
            row_text.append(day + datetime.timedelta(days = 17))
            data.append(row_text)
        data_df = pd.DataFrame(data, columns=headers)
        master_df = master_df.append(data_df)

In [64]:
master_df.to_csv('data/pitching/past_3games_master.csv')

In [188]:
master_df = pd.DataFrame(columns = headers)
for file in os.listdir('data/pitching/'):
    if file.startswith('past_3games'):
        master_df = master_df.append(pd.read_csv('data/pitching/'+file, index_col = 0))
master_df.to_csv('data/pitching/past_3games_master.csv')

## Batted Balls

In [286]:
binary = FirefoxBinary(r'C:\Program Files (x86)\Mozilla Firefox\firefox.exe')
browser = webdriver.Firefox(firefox_binary=binary)
url_list = ['http://www.fangraphs.com/leaderssplits.aspx?splitArr=&strgroup=season&statgroup=3&startDate=2016-04-08&endDate=',
       str(day),
        '&filter=&position=P&statType=player&autoPt=true&sort=18,-1&pg=',
       str(page)]
url = "".join(url_list)
browser.get(url)   
table = None

#pull html table from first page
while table == None:
    soup = BS(browser.page_source, "lxml")
    table = soup.find(attrs = {'class':"table-splits"})
    sleep(1)
for tr in table.find('thead').findAll('tr'):
    headers = [c.text for c in tr.findAll('th')]
headers.append('Date')

In [66]:
headers = ['#',
 'Season',
 'Name',
 'Team',
 'IP',
 'TBF',
 'GB/FB',
 'LD%',
 'GB%',
 'FB%',
 'IFFB%',
 'HR/FB',
 'IFH%',
 'BUH%',
 'Pull%',
 'Cent%',
 'Oppo%',
 'Soft%',
 'Med%',
 'Hard%',
 'Date']

In [67]:
# Create list of days to iterate over
start = datetime.date(year = 2015, month = 4, day = 5)
end = datetime.date(year = 2015, month = 10, day = 4)
date_range = []
while start <= end:
    date_range.append(start)
    start += datetime.timedelta(days = 1)

# Create master dataframe and open firefox
master_df = pd.DataFrame(columns = headers)
binary = FirefoxBinary(r'C:\Program Files (x86)\Mozilla Firefox\firefox.exe')
browser = webdriver.Firefox(firefox_binary=binary)

# iterate through all days in range, and enter in url
for day in date_range:
    page = 0
    url_list = ['http://www.fangraphs.com/leaderssplits.aspx?splitArr=&strgroup=season&statgroup=3&startDate=2015-04-05&endDate=',
           str(day),
            '&filter=&position=P&statType=player&autoPt=false&sort=18,-1&pg=',
           str(page)]
    url = "".join(url_list)
    browser.get(url)   
    table = None
    
    #pull html table from first page
    while table == None:
        soup = BS(browser.page_source, "lxml")
        table = soup.find(attrs = {'class':"table-splits"})
        sleep(1)
    body = table.find('tbody')
    if body != None:
        rows = body.findAll('tr')
    else:
        continue
    
    #parses data and appends to master dataframe
    data = []
    for tr in rows:
        cols = tr.findAll('td')
        row_text = [c.text for c in cols]
        row_text.append(day)
        data.append(row_text)
    data_df = pd.DataFrame(data, columns=headers)
    master_df = master_df.append(data_df)
    
    #close stupid ad getting in the way
    
    
    #iterate through all pages for given day
    page = int(soup.find('span', attrs = {'class':"table-control-total"}).text)
    for p in range(1,page):
        
        #scroll to bottom of page to prevent ad intereference
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        #click next button
        browser.find_element_by_class_name('material-icons' and 'next').click()
        sleep(1)
        
        #parses data and appends to master dataframe
        data = []
        soup = BS(browser.page_source, "lxml")
        table = soup.find(attrs = {'class':"table-splits"})
        body = table.find('tbody')
        if body != None:
            rows = body.findAll('tr')
        else:
            continue
        for tr in rows:
            cols = tr.findAll('td')
            row_text = [c.text for c in cols]
            row_text.append(day)
            data.append(row_text)
        data_df = pd.DataFrame(data, columns=headers)
        master_df = master_df.append(data_df)

In [68]:
master_df.to_csv('data/pitching/season_batted_balls_master.csv')

In [313]:
master_df = pd.DataFrame(columns = headers)
for file in os.listdir('data/pitching/'):
    if file.startswith('season_batted_balls'):
        master_df = master_df.append(pd.read_csv('data/pitching/'+file, index_col = 0))
master_df.to_csv('data/pitching/season_batted_balls_master.csv')

## Past Three Starts

In [69]:
# Create list of days to iterate over
start = datetime.date(year = 2015, month = 3, day = 19)
end = datetime.date(year = 2015, month = 9, day = 17)
date_range = []
while start <= end:
    date_range.append(start)
    start += datetime.timedelta(days = 1)

# Create master dataframe and open firefox
master_df = pd.DataFrame(columns = headers)
binary = FirefoxBinary(r'C:\Program Files (x86)\Mozilla Firefox\firefox.exe')
browser = webdriver.Firefox(firefox_binary=binary)

# iterate through all days in range, and enter in url
for day in date_range:
    page = 0
    url_list = ['http://www.fangraphs.com/leaderssplits.aspx?splitArr=&strgroup=season&statgroup=3&startDate=',
           str(day),'&endDate=',str(day + datetime.timedelta(days = 17)),
            '&filter=&position=P&statType=player&autoPt=false&sort=18,-1&pg=',
           str(page)]
    url = "".join(url_list)
    browser.get(url)   
    table = None
    
    #pull html table from first page
    while table == None:
        soup = BS(browser.page_source, "lxml")
        table = soup.find(attrs = {'class':"table-splits"})
        sleep(1)
    body = table.find('tbody')
    if body != None:
        rows = body.findAll('tr')
    else:
        continue
    
    #parses data and appends to master dataframe
    data = []
    for tr in rows:
        cols = tr.findAll('td')
        row_text = [c.text for c in cols]
        row_text.append(day + datetime.timedelta(days = 17))
        data.append(row_text)
    data_df = pd.DataFrame(data, columns=headers)
    master_df = master_df.append(data_df)
    
    #iterate through all pages for given day
    page = int(soup.find('span', attrs = {'class':"table-control-total"}).text)
    for p in range(1,page):
        
        #scroll to bottom of page to prevent ad intereference
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")        
        
        browser.find_element_by_class_name('material-icons' and 'next').click()
        sleep(1)
        
        #parses data and appends to master dataframe
        data = []
        soup = BS(browser.page_source, "lxml")
        table = soup.find(attrs = {'class':"table-splits"})
        body = table.find('tbody')
        if body != None:
            rows = body.findAll('tr')
        else:
            continue
        for tr in rows:
            cols = tr.findAll('td')
            row_text = [c.text for c in cols]
            row_text.append(day + datetime.timedelta(days = 17))
            data.append(row_text)
        data_df = pd.DataFrame(data, columns=headers)
        master_df = master_df.append(data_df)

In [70]:
master_df.to_csv('data/pitching/past_3games_batted_balls_master.csv')

In [356]:
master_df = pd.DataFrame(columns = headers)
for file in os.listdir('data/pitching/'):
    if file.startswith('past_3games_batted_balls'):
        master_df = master_df.append(pd.read_csv('data/pitching/'+file, index_col = 0))
master_df.to_csv('data/pitching/past_3games_batted_balls_master.csv')

## Bullpen
### Season

In [357]:
binary = FirefoxBinary(r'C:\Program Files (x86)\Mozilla Firefox\firefox.exe')
browser = webdriver.Firefox(firefox_binary=binary)
url_list = ['http://www.fangraphs.com/leaderssplits.aspx?splitArr=43&strgroup=season&statgroup=2&startDate=2016-04-03&endDate=',
       str(day),
        '&filter=&position=P&statType=team&autoPt=false&sort=16,-1&pg=',
       str(page)]
url = "".join(url_list)
browser.get(url)   
table = None

#pull html table from first page
while table == None:
    soup = BS(browser.page_source, "lxml")
    table = soup.find(attrs = {'class':"table-splits"})
    sleep(1)
for tr in table.find('thead').findAll('tr'):
    headers = [c.text for c in tr.findAll('th')]
headers.append('Date')

In [71]:
headers = ['#',
 'Season',
 'Team',
 'IP',
 'TBF',
 'K/9',
 'BB/9',
 'K/BB',
 'HR/9',
 'K%',
 'BB%',
 'K-BB%',
 'AVG',
 'WHIP',
 'BABIP',
 'LOB%',
 'FIP',
 'xFIP',
 'Date']

In [72]:
# Create list of days to iterate over
start = datetime.date(year = 2015, month = 4, day = 5)
end = datetime.date(year = 2015, month = 10, day = 4)
date_range = []
while start <= end:
    date_range.append(start)
    start += datetime.timedelta(days = 1)

# Create master dataframe and open firefox
master_df = pd.DataFrame(columns = headers)
binary = FirefoxBinary(r'C:\Program Files (x86)\Mozilla Firefox\firefox.exe')
browser = webdriver.Firefox(firefox_binary=binary)

# iterate through all days in range, and enter in url
for day in date_range:
    page = 0
    url_list = ['http://www.fangraphs.com/leaderssplits.aspx?splitArr=43&strgroup=season&statgroup=2&startDate=2015-04-05&endDate=',
           str(day),
            '&filter=&position=P&statType=team&autoPt=false&sort=16,-1&pg=',
           str(page)]
    url = "".join(url_list)
    browser.get(url)   
    table = None
    
    #pull html table from first page
    while table == None:
        soup = BS(browser.page_source, "lxml")
        table = soup.find(attrs = {'class':"table-splits"})
        sleep(1)
    body = table.find('tbody')
    if body != None:
        rows = body.findAll('tr')
    else:
        continue
    
    #parses data and appends to master dataframe
    data = []
    for tr in rows:
        cols = tr.findAll('td')
        row_text = [c.text for c in cols]
        row_text.append(day)
        data.append(row_text)
    data_df = pd.DataFrame(data, columns=headers)
    master_df = master_df.append(data_df)
    
    #close stupid ad getting in the way
    
    
    #iterate through all pages for given day
    page = int(soup.find('span', attrs = {'class':"table-control-total"}).text)
    for p in range(1,page):
        
        #scroll to bottom of page to prevent ad intereference
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        #click next button
        browser.find_element_by_class_name('material-icons' and 'next').click()
        sleep(1)
        
        #parses data and appends to master dataframe
        data = []
        soup = BS(browser.page_source, "lxml")
        table = soup.find(attrs = {'class':"table-splits"})
        body = table.find('tbody')
        if body != None:
            rows = body.findAll('tr')
        else:
            continue
        for tr in rows:
            cols = tr.findAll('td')
            row_text = [c.text for c in cols]
            row_text.append(day)
            data.append(row_text)
        data_df = pd.DataFrame(data, columns=headers)
        master_df = master_df.append(data_df)

In [74]:
master_df.to_csv('data/pitching/bullpen_season_master.csv')

In [369]:
master_df = pd.DataFrame(columns = headers)
for file in os.listdir('data/pitching/'):
    if file.startswith('bullpen_season'):
        master_df = master_df.append(pd.read_csv('data/pitching/'+file, index_col = 0))
master_df.to_csv('data/pitching/bullpen_season_master.csv')

# Past 7 Days

In [75]:
# Create list of days to iterate over
start = datetime.date(year = 2015, month = 4, day = 5)
end = datetime.date(year = 2015, month = 9, day = 27)
date_range = []
while start <= end:
    date_range.append(start)
    start += datetime.timedelta(days = 1)

# Create master dataframe and open firefox
master_df = pd.DataFrame(columns = headers)
binary = FirefoxBinary(r'C:\Program Files (x86)\Mozilla Firefox\firefox.exe')
browser = webdriver.Firefox(firefox_binary=binary)

# iterate through all days in range, and enter in url
for day in date_range:
    page = 0
    url_list = ['http://www.fangraphs.com/leaderssplits.aspx?splitArr=43&strgroup=season&statgroup=2&startDate=',
           str(day),'&endDate=',str(day + datetime.timedelta(days = 7)),
            '&filter=&position=P&statType=team&autoPt=false&sort=16,-1&pg=',
           str(page)]
    url = "".join(url_list)
    browser.get(url)   
    table = None
    
    #pull html table from first page
    while table == None:
        soup = BS(browser.page_source, "lxml")
        table = soup.find(attrs = {'class':"table-splits"})
        sleep(1)
    body = table.find('tbody')
    if body != None:
        rows = body.findAll('tr')
    else:
        continue
    
    #parses data and appends to master dataframe
    data = []
    for tr in rows:
        cols = tr.findAll('td')
        row_text = [c.text for c in cols]
        row_text.append(day + datetime.timedelta(days = 7))
        data.append(row_text)
    data_df = pd.DataFrame(data, columns=headers)
    master_df = master_df.append(data_df)
    
    #iterate through all pages for given day
    page = int(soup.find('span', attrs = {'class':"table-control-total"}).text)
    for p in range(1,page):
        
        #scroll to bottom of page to prevent ad intereference
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        browser.find_element_by_class_name('material-icons' and 'next').click()
        sleep(1)
        
        #parses data and appends to master dataframe
        data = []
        soup = BS(browser.page_source, "lxml")
        table = soup.find(attrs = {'class':"table-splits"})
        body = table.find('tbody')
        if body != None:
            rows = body.findAll('tr')
        else:
            continue
        for tr in rows:
            cols = tr.findAll('td')
            row_text = [c.text for c in cols]
            row_text.append(day + datetime.timedelta(days = 7))
            data.append(row_text)
        data_df = pd.DataFrame(data, columns=headers)
        master_df = master_df.append(data_df)

In [76]:
master_df.to_csv('data/pitching/bullpen_past7_master.csv')

In [376]:
master_df = pd.DataFrame(columns = headers)
for file in os.listdir('data/pitching/'):
    if file.startswith('bullpen_past7'):
        master_df = master_df.append(pd.read_csv('data/pitching/'+file, index_col = 0))
master_df.to_csv('data/pitching/bullpen_past7_master.csv')